## CT5141 Optimisation Assignment 2

#### Jiarong Li, Aidan Wallace

#### ID 20230033, 22225668

In [1]:
import time
import random
import numpy as np
import pandas as pd
import knapsack as ks
import matplotlib.pyplot as plt
from ortools.algorithms import pywrapknapsack_solver

### 0-1 Knapsack Problem

The objective of the knapsack problem is to pack items with a given weight and value into a container of given capacity so as to maximise the value of the items in the container while not exceeding the maximum weight capacity.

The knapsack problem we are given can be defined as:

Maximise $ \sum^{n}_{i=1} p_i x_i $

Subject to $ \sum^{n}_{i=1} w_i x_i \leq W $

$ x \in \{ 0, 1 \} \  \forall \  i \in \{1, 2, \cdots, n\} $

where,

- $p_i$ is the value of item $i$
- $w_i$ is the weight of item $i$
- $x_i$ is a binary decision variable to determine if item $i$ is selected
- $W$ is the total capacity of the container

In [2]:
# Read data using function read_knapsack_data from knapsack.py
n_items, f = ks.read_knapsack_data('knapsack_data/knapPI_1_500_1000_1', verbose=False)

ks_data = pd.read_table('knapsack_data/knapPI_1_500_1000_1', sep='\s+', names=['values', 'weights'])
max_weight = ks_data.iloc[0, 1]
values = ks_data.iloc[1:, 0].to_numpy()
weights = ks_data.iloc[1:, 1].to_numpy()

print(f'Total # items: {n_items}')
print(f'Max weight capacity: {max_weight}')

Total # items: 500
Max weight capacity: 2543


### Finding Optimal Solutions

We implement a Dynamic Programming (DP) algorithm from Google's OR-Tools package in Python to determine an optimal solution (not neccessarily the only optimal solution) to our knapsack problem. From this solution, we can compare various approximation algorithms to determine how close they get to finding the optimal packed value. 

The 0-1 knapsack problem problem is weakly NP-complete. Using Dynamic Programming to solve our knapsack problem will provide an optimal solution in pseudo-polynomial time, $O(nW)$. (https://en.wikipedia.org/wiki/Pseudo-polynomial_time)

#### Dynamic Programming (DP)

Dynamic programming is an optimisation technique that uses recursion to break down a problem into smaller subproblems. DP is suitable for problems of this nature, where the subproblems have some relation to the objective of the larger problem. Dynamic programming for the 0-1 Knapsack problem works as follows:

Let $x_1, x_2, \cdots, x_i$ be a subset of the total items $N$ to be packed. If $P(i, c)$ is the profit from the selection of $i$ items (where $c$ is the remaining capacity out of a total capacity $M$) then we can define the following recursion,

$$
P(i, c)= \begin{cases}0 & \text { if } i=0 \text { or } c \leq 0 \\ \max \left(P(i-1, c), P\left(i-1, c-w_i\right)+p_i\right) & \text { if } w_i \leq c \\ P(i-1, c) & \text { otherwise }\end{cases}
$$

We create a table of subset values by computing all values of $P(i, c)$ for $0 \leq i \leq N$ and $0 \leq c \leq M$ to find the optimal value $P(N, M)$. Since this method only gives us the total value of the solution, we then need to implement a method of 'backtracking' from our total value to find the items that we place into the knapsack.

In [3]:
# Dynamic programming (DP) algorithm for finding optimal solution (OR-Tools)
solver = pywrapknapsack_solver.KnapsackSolver(pywrapknapsack_solver.KnapsackSolver.KNAPSACK_DYNAMIC_PROGRAMMING_SOLVER, '0_1_Knapsack')

solver.Init(list(values), [list(weights)], [max_weight])

start = time.time()
computed_value = solver.Solve()
end = time.time()
time_elapsed = end - start

total_weight = 0
packed_items = []
packed_weights = []
for i in range(len(values)):
	if solver.BestSolutionContains(i):
		packed_items.append(i)
		packed_weights.append(weights[i])
		total_weight += weights[i]

print('Total value =', computed_value)
print('Total weight:', total_weight)
print('Number of packed items:', len(packed_items))

dp_solution = pd.DataFrame({'index': [0], 'method': ['DP'], 'random_seed': [None], 'budget': [None], 'customisations': [None], 'hyperparameters': [None], 'best_solution': [packed_items], 'best_value': [computed_value], 'best_weight': [total_weight], 'elapsed_time': time_elapsed})

Total value = 28857
Total weight: 2543
Number of packed items: 42


In [4]:
# Genetric Algorithm (GA) Implementation
class GA:
    def __init__(self, f, popsize, ngens, tsize, seed) -> None:
        self.f = f
        self.popsize = popsize
        self.ngens = ngens
        self.tsize = tsize
        self.seed = seed
    
    def run(self, genome_length):
        
        random.seed(self.seed)
        np.random.seed(self.seed)
    
        # make initial population, evaluate fitness
        inds = [self.init(7, genome_length) for _ in range(self.popsize)]
        pop = [(self.f(x), x) for x in inds]
        
        for _ in range(1, self.ngens):
            # make a new population, evaluate fitness
            inds = [self.bitflips(self.uniform_crossover(self.tournament_select(pop), self.tournament_select(pop)))
                    for _ in range(self.popsize)]
            pop = [(self.f(x), x) for x in inds]
            
        return max(pop)

    def tournament_select(self, pop):
        candidates = random.sample(pop, self.tsize)
        # The winner is the index of the individual with max fitness, ie c[0]
        winner = max(candidates, key=lambda c: c[0])
        return winner[1] # return the genome only
    
    def init(self, init_slice, genome_length):
        inputs = [0 for _ in range(genome_length)]
        inputs[:init_slice] = [random.randrange(2) for _ in range(init_slice)]
        return inputs
    
    def bitflips(self, p):
        return [(1 - pi if np.random.random() < 1.0 / len(p) else pi) for pi in p]
    
    def uniform_crossover(self, p1, p2):
        return [(p1i if np.random.random() < 0.5 else p2i) for p1i, p2i in zip(p1, p2)]


### Genetic Algorithms (GA)

We use the GA approach to find an approximate solution for the knapsack problem. Genetic Algorithm is a metaheuristic optimisation with genetic operators (initialisation, mutation, crossover) and selection operator to guide search. However, there are many infeasible solutions during searching. In this case, we apply two solutions to improve the performance of finding the objectives with GA.

We first set the init() function with the positions which are most likely to obtain the objective.

We then apply the objective penalty 'f_reflect()' to individuals who are over-weight to obtain the objective.

In [5]:
def f_reflect(x):
    weight_sum, value_sum = ks.eval_knapsack(weights, values, max_weight, x)
    
    excess = weight_sum - max_weight
    
    if excess > 0:
        return -excess # over-weight, reflect the value
    else:
        return value_sum

### Simulated Annealing (SA)

Simulated Annealing is a method developed to solve combinatorial optimisation problems. It is based on the physical process of the heating of materials past their critical temperature (liquid - high energy state) and then cooled according to a cooling schedule until they reach a minimum energy state (solid configuration). This proccess is known as annealing. 

The simulated annealing algorithm mimicks this process by starting off at a high temperature where the algorithm is able to energetically explore the search space through random generation of neighbour solutions using some suitably defined mechanism. At each step, the neighbours value is calculated and accepted if it is greater than the current value, else it is accepted with a certain probability which is a function of the current temperature. The temperature is slowly decreased so that the algorithm takes less disimproving moves as it runs. Simulated annealing avoids getting stuck at local minima by accepting some disimproving moves. The acceptance probability is given by,

$$
p_k=\exp \left[-\frac{\Delta E}{T_k}\right]
$$

where $\Delta E$ is the change in value from the current to the the proposed solution and $T_k$ is the temperature at iteration $k$.

Different cooling schedules exist for simulated annealing. The difficultly with running this algorithm is in finding the optimal combination of starting temperature and cooling schedule as no single rule has been defined for the general optimisation problem. 

- linear cooling:
$$
T_k=T_0-\alpha k, \quad \alpha=\frac{T_0-T_f}{N}
$$
- exponential multiplicative cooling, also referred to as geometric cooling (Kirkpatrick et al.):
$$
T_k=T_0 \alpha^k, \quad 0.7 \leq \alpha \leq 0.99
$$
- logarithmic multiplicative cooling (Aarts and Korst):
$$
T_k=\frac{T_0}{1+\alpha \log (1+k)}, \quad \alpha>1 ;
$$

In [6]:
# Simulated Annealing (SA) Implementation
class SA:
	def __init__(self, capacity, t0, t_min, alpha, epochs, max_iter, seed, schedule='geometric', verbose=False) -> None:
		'''
		Simulated annealing implementation for solving the 0-1 knapsack problem. Maximise the total value (profit) of packed items while meeting the total weight constraint.

		capacity: maximum capacity of knapsack
		t0: initial temperature
		t_min: minimum temperature (stopping criteria)
		alpha: cooling coefficient
		epochs: number of iterations to run at each temperature
		max_iter: maximum number of iterations
		seed: random seed
		schedule: cooling schedule use for reducing temperature {linear, geometric, logarithmic}
		verbose: print out results during fitting of model
		'''
		self.capacity = capacity
		self.t0 = t0
		self.t_min = t_min
		self.alpha = alpha
		self.epochs = epochs
		self.max_iter = max_iter
		self.seed = seed
		self.schedule = schedule
		self.verbose = verbose
	
	def fit(self, values, weights):
		'''
		'Run' method for simulated annealing algorithm.

		values: 
		'''
		random.seed(self.seed)
		np.random.seed(self.seed)
		
		self.values = np.array(values)
		self.weights = np.array(weights)
		self.n = len(values)

		t = self.t0
		accepted = 0
		current = self.initial_solution()
		best = []

		i = 0
		while t > self.t_min and i < self.max_iter:
			if self.verbose:
				print(f'iteration {i}')
				print('temperature:', t)
			
			for _ in range(self.epochs):
				# Propose new neighbour solution
				proposal = self.get_proposal(current)

				# Change in total profit (energy)
				delta = self.get_profit(proposal) - self.get_profit(current) # E_proposal - E_current
				
				if delta > 0: # accept change if leads to improvement in total profit
					current = proposal
					# accepted += 1
				else:
					# Accept proposal with probability p = e^{(current profit - proposal profit)/t}
					acceptance_prob = min(1, np.exp(-delta/t))
					if np.random.random() <= acceptance_prob:
						current = proposal
						accepted += 1
					
				# determine if proposal is more profitable than best solution found so far
				if self.get_profit(proposal) > self.get_profit(best):
					best = proposal
				
			t = self.cool(i)
			i += 1
			
		packed_items = np.where(best)
		
		if self.verbose:
			print('Number of iterations completed:', i)
			print('Total weight', self.get_weight(best))
			print('Total profit', self.get_profit(best))
			print(f'acceptance rate: {accepted/(self.max_iter*self.epochs)}')

		return packed_items, self.get_weight(best), self.get_profit(best), accepted/(self.max_iter*self.epochs)
	
	def cool(self, i):
		'''
		Cooling schedule for temperature parameter
		'''
		if self.schedule == 'geometric':
			return self.t0 * self.alpha**i
		elif self.schedule == 'linear':
			return self.t0 - self.alpha*i
		elif self.schedule == 'logarithmic':
			return self.t0/(1+self.alpha*np.log(1+self.max_iter))
		else: 
			return
	
	def initial_solution(self):
		'''
		Find a feasible initial solution. Returns a list of indices of items to be packed.
		'''
		while True:
			x = np.array(np.zeros(self.n), dtype=bool)
			true_indices = np.random.randint(0, self.n, 2) # list index positions to change to True (True = select item)
			x[true_indices] = True
			if self.validate(x) == True:
				return x

	def get_proposal(self, x):
		'''
		Function to calculate new propsal solution which is neighbour of current solution. Randomly add or remove one item from selection.

		x: list of indices of current items to be packed (current solution)
		'''
		while True:
			proposal = x.copy()
			index = np.random.randint(0, self.n)
			proposal[index] = False if proposal[index] == True else True
			if self.validate(proposal) == True:
				return proposal
	
	def validate(self, x):
		'''
		Helper function to check if total weight of items to be packed is less than or equal to maximum knapsack capacity.

		x: list of boolean values {0, 1}, where 1 indicates an item to be packed
		'''
		return True if self.get_weight(x) <= self.capacity else False
	
	def get_weight(self, x):
		'''
		Calculate total weight of selected items

		x: list of boolean values {0, 1}, where 1 indicates an item to be packed
		'''
		return np.sum(self.weights[x])


	def get_profit(self, x):
		'''
		Calculate total profit of selected items

		x: list of boolean values {0, 1}, where 1 indicates an item to be packed
		'''
		return np.sum(self.values[x])

In [7]:
# Run models 5 times each
random_seeds = [0, 1, 2, 3, 4]
solutions = [dp_solution]

budget = 50000
# GA Hyperparameters
popsize = 100
ngens = budget // popsize
tsize = 5

# SA Hyperparameters
t0 = 100
t_min = 0.000001
alpha = 0.998
epochs = 10
max_iter = budget // epochs

i = 1
for r in random_seeds:
	
	# Run GA (with f)
	ga1 = GA(f=f, popsize=popsize, ngens=ngens, tsize=tsize, seed=r)

	ga1_start = time.time()
	ga1_sol = ga1.run(genome_length=n_items)
	ga1_end = time.time()

	ga1_best_weight, ga1_best_value = ks.eval_knapsack(weights=weights, values=values, max_weight=max_weight, x=ga1_sol[1])

	ga1_time_elapsed = ga1_end - ga1_start

	temp_ga1_solution = pd.DataFrame({'index': [i], 'method': ['GA'], 'random_seed': [r], 'budget': [budget], 'customisations': ['Initial solution has 7 items selected'], 'hyperparameters': [{'population_size': popsize, 'tournament_size': tsize, 'mutation_rate': 1/n_items}], 'best_solution': [np.where(ga1_sol[1])], 'best_value': [ga1_best_value], 'best_weight': [ga1_best_weight], 'elapsed_time': ga1_time_elapsed})

	solutions.append(temp_ga1_solution)
	i += 1

	# Run GA (with f_reflect)
	ga2 = GA(f=f_reflect, popsize=popsize, ngens=ngens, tsize=tsize, seed=r)

	ga2_start = time.time()
	ga2_sol = ga2.run(genome_length=n_items)
	ga2_end = time.time()

	ga2_best_weight, ga2_best_value = ks.eval_knapsack(weights=weights, values=values, max_weight=max_weight, x=ga2_sol[1])

	ga2_time_elapsed = ga2_end - ga2_start

	temp_ga2_solution = pd.DataFrame({'index': [i], 'method': ['GA (Penalised Fitness)'], 'random_seed': [r], 'budget': [budget], 'customisations': [['Objective function used is f_reflect', 'Initial solution contains 7 items']], 'hyperparameters': [{'population_size': popsize, 'tournament_size': tsize, 'mutation_rate': 1/n_items}], 'best_solution': [np.where(ga2_sol[1])], 'best_value': [ga2_best_value], 'best_weight': [ga2_best_weight], 'elapsed_time': [ga2_time_elapsed]})

	solutions.append(temp_ga2_solution)
	i += 1

	# Run SA (Geometric)
	sa1 = SA(capacity=max_weight, t0=t0, t_min=t_min, alpha=alpha, epochs=epochs, max_iter=max_iter, seed=r, schedule='geometric', verbose=False)

	sa1_start = time.time()
	sa1_items, sa1_best_weight, sa1_best_value, acceptance_rate = sa1.fit(values=values, weights=weights)
	sa1_end = time.time()

	sa1_time_elapsed = sa1_end - sa1_start

	temp_sa1_solution = pd.DataFrame({'index': [i], 'method': ['SA (Geometric)'], 'random_seed': [r], 'budget': [budget], 'customisations': [['Initial solution contains 2 items']], 'hyperparameters': [{'Initial Temperature': t0, 'Minimum Temperature': t_min, 'Cooling Rate': alpha, 'cooling_schedule': 'geometric'}], 'best_solution': [sa1_items], 'best_value': [sa1_best_value], 'best_weight': [sa1_best_weight], 'elapsed_time': [sa1_time_elapsed]})

	solutions.append(temp_sa1_solution)
	i += 1

	# Run SA (Logarithmic)
	sa2 = SA(capacity=max_weight, t0=t0, t_min=t_min, alpha=alpha, epochs=epochs, max_iter=max_iter, seed=r, schedule='logarithmic', verbose=False)

	sa2_start = time.time()
	sa2_items, sa2_best_weight, sa2_best_value, acceptance_rate = sa2.fit(values=values, weights=weights)
	sa2_end = time.time()

	sa2_time_elapsed = sa2_end - sa2_start

	temp_sa2_solution = pd.DataFrame({'index': [i], 'method': ['SA (Logarithmic)'], 'random_seed': [r], 'budget': [budget], 'customisations': [['Initial solution contains 2 items']], 'hyperparameters': [{'Initial Temperature': t0, 'Minimum Temperature': t_min, 'Cooling Rate': alpha, 'cooling_schedule': 'logarithmic'}], 'best_solution': [sa2_items], 'best_value': [sa2_best_value], 'best_weight': [sa2_best_weight], 'elapsed_time': [sa2_time_elapsed]})

	solutions.append(temp_sa2_solution)
	i += 1

	

C:\Windows\Temp\ipykernel_18016\1918942543.py:63: RuntimeWarning: overflow encountered in exp
  acceptance_prob = min(1, np.exp(-delta/t))


In [8]:
# Dataframe to store solutions
solution_df =pd.DataFrame({'index': [], 'method': [], 'random_seed': [], 'budget': [], 'customisations': [], 'hyperparameters': [], 'best_solution': [], 'best_value': [], 'best_weight': [], 'elapsed_time': []})

solution_df = pd.concat([solution_df]+solutions, axis=0, ignore_index=True)
solution_df.sort_values(by=['best_value'], ascending=False)

,index,method,random_seed,budget,customisations,hyperparameters,best_solution,best_value,best_weight,elapsed_time
0,0.0,DP,None,None,None,None,"[6, 10, 12, 13, 23, 25, 32, 37, 38, 48, 53, 60...",28857.0,2543.0,0.018949
5,5.0,GA,1,50000,Initial solution has 7 items selected,"{'population_size': 100, 'tournament_size': 5,...","([6, 10, 13, 23, 32, 37, 48, 53, 60, 121, 134,...",27538.0,2526.0,35.416254
14,14.0,GA (Penalised Fitness),3,50000,"[Objective function used is f_reflect, Initial...","{'population_size': 100, 'tournament_size': 5,...","([6, 10, 13, 23, 30, 32, 37, 38, 48, 53, 60, 1...",27535.0,2542.0,35.340457
6,6.0,GA (Penalised Fitness),1,50000,"[Objective function used is f_reflect, Initial...","{'population_size': 100, 'tournament_size': 5,...","([6, 10, 12, 13, 23, 25, 30, 32, 37, 38, 48, 6...",27236.0,2541.0,35.608738
18,18.0,GA (Penalised Fitness),4,50000,"[Objective function used is f_reflect, Initial...","{'population_size': 100, 'tournament_size': 5,...","([6, 10, 12, 30, 32, 37, 38, 48, 53, 60, 121, ...",26792.0,2516.0,35.110118
17,17.0,GA,4,50000,Initial solution has 7 items selected,"{'population_size': 100, 'tournament_size': 5,...","([6, 10, 23, 25, 27, 30, 32, 37, 48, 53, 60, 1...",26787.0,2518.0,35.665585
1,1.0,GA,0,50000,Initial solution has 7 items selected,"{'population_size': 100, 'tournament_size': 5,...","([6, 10, 13, 23, 25, 32, 37, 38, 48, 53, 60, 1...",26117.0,2538.0,36.253032
9,9.0,GA,2,50000,Initial solution has 7 items selected,"{'population_size': 100, 'tournament_size': 5,...","([3, 6, 10, 12, 23, 30, 32, 37, 38, 48, 53, 60...",26059.0,2531.0,35.366387
2,2.0,GA (Penalised Fitness),0,50000,"[Objective function used is f_reflect, Initial...","{'population_size': 100, 'tournament_size': 5,...","([3, 6, 10, 13, 23, 32, 37, 38, 48, 53, 60, 12...",25925.0,2538.0,35.543942
13,13.0,GA,3,50000,Initial solution has 7 items selected,"{'population_size': 100, 'tournament_size': 5,...","([3, 6, 10, 13, 23, 32, 37, 38, 48, 53, 60, 10...",25583.0,2532.0,35.340432


The solutions table above shows the optimal value of packed items (the value found using dynamic programming) is 28857 with the total weight of items equal to the maximum weight constraint. The closest approximation to the optimal solution is 27538 which comes from running our GA with tournament selectection, uniform crossover and bitflip mutation, initial population size of 100, tournament size of 5 and mutation rate of 0.2%.  In general, the GA's performed better than simulated annealing for this task. The best simulated annealing solution is a value of 21053. Simulated annealing was run with a geometric and logarithmic cooling schedule. Average run times are given using an Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz with 8gb RAM. The DP algorithm ran in 0.02 seconds, the average run time for GA is aprrox. 37 seconds and for SA the average run time is approx. 10 seconds. The average value of the knapsack for GA solutions is 26416.8 and for GA with penalised fitness the average is 26081.4. For SA, the average best value is 20164.8 for both cooling chedules.

In [9]:
solution_df.groupby('method').mean()[['best_value', 'best_weight', 'elapsed_time']]

,best_value,best_weight,elapsed_time
method,,,
DP,28857.0,2543.0,0.018949
GA,26416.8,2529.0,35.608338
GA (Penalised Fitness),26081.4,2535.0,35.393128
SA (Geometric),20164.8,2494.4,8.870280
SA (Logarithmic),20164.8,2494.4,8.829773
